In [1]:
import numpy as np
import math
import os
import time
import logging
from hyperopt.plotting import *
from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials, base, rand
from data.data_loader import Dataset
from misc.preferences import PREFERENCES
from misc.run_configuration import from_hyperopt, OutputLayerType, LearningSchedulerType, OptimizerType, default_params
from misc import utils
from misc.hyperopt_space import *

from optimizer import get_optimizer
from criterion import NllLoss, LossCombiner
from models.transformer.encoder import TransformerEncoder
from models.jointAspectTagger import JointAspectTagger
from trainer.train import Trainer
import pprint
import argparse
import pickle
import traceback


In [2]:
POSSIBLE_DATASET_VALUES = ['germeval', 'organic', 'coNLL-2003', 'amazon']


def load_model(dataset, rc, experiment_name):
    transformer = TransformerEncoder(dataset.source_embedding,
                                        hyperparameters=rc)

    # NER or ABSA-task?
    if rc.task == 'ner':
        from models.transformer_tagger import TransformerTagger
        from models.output_layers import SoftmaxOutputLayer
        loss = NllLoss(dataset.target_size, dataset.class_weights[0])
        softmax = SoftmaxOutputLayer(rc.model_size, dataset.target_size)
        model = TransformerTagger(transformer, softmax)

    else:
        from models.jointAspectTagger import JointAspectTagger
        loss = LossCombiner(4, dataset.class_weights, NllLoss)
        model = JointAspectTagger(transformer, rc, 4, 20, dataset.target_names)


    optimizer = get_optimizer(model, rc)
    trainer = Trainer(
                        model,
                        loss,
                        optimizer,
                        rc,
                        dataset,
                        experiment_name,
                        enable_tensorboard=False,
                        verbose=False)
    return trainer

def load_dataset(rc, logger, task):
    dataset = Dataset(
            task,
            logger,
            rc,
            source_index=PREFERENCES.source_index,
            target_vocab_index=PREFERENCES.target_vocab_index,
            data_path=PREFERENCES.data_root,
            train_file=PREFERENCES.data_train,
            valid_file=PREFERENCES.data_validation,
            test_file=PREFERENCES.data_test,
            file_format=PREFERENCES.file_format,
            init_token=None,
            eos_token=None
        )
    dataset.load_data(dsl, verbose=False)
    return dataset

def objective(parameters):
    run_time = time.time()

    utils.reset_loggers()
    experiment_name = utils.create_loggers(experiment_name=main_experiment_name)
    logger = logging.getLogger(__name__)
    dataset_logger = logging.getLogger('data_loader')

    # generate hp's from parameters
    try:
        rc = from_hyperopt(parameters, use_cuda, model_size=300, early_stopping=5, num_epochs=35, log_every_xth_iteration=-1, language=PREFERENCES.language)
    except Exception as err:
        print('Could not convert params: ' + str(err))
        logger.exception("Could not load parameters from hyperopt configuration: " + parameters)
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }
    logger.info('New Params:')
    logger.info(rc)
    print('\n\n#########################################################################')
    print(rc)

    logger.debug('Load dataset')
    try:
        dataset = load_dataset(rc, dataset_logger, rc.task)
    except Exception as err:
        print('Could not load dataset: ' + str(err))
        logger.exception("Could not load dataset")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }
    logger.debug('dataset loaded')
    logger.debug('Load model')

    try:
        trainer = load_model(dataset, rc, experiment_name)
    except Exception as err:
        print('Could not load model: ' + str(err))
        logger.exception("Could not load model")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }

    logger.debug('model loaded')

    logger.debug('Begin training')
    model = None
    try:
        result = trainer.train(use_cuda=rc.use_cuda, perform_evaluation=False)
        model = result['model']
    except Exception as err:
        print('Exception while training: ' + str(err))
        logger.exception("Could not complete iteration")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }

    if math.isnan(trainer.get_best_loss()):
        print('Loss is nan')
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }

    # perform evaluation and log results
    result = None
    try:
        result = trainer.perform_final_evaluation(use_test_set=True, verbose=False)
    except Exception as err:
        logger.exception("Could not complete iteration evaluation.")
        print('Could not complete iteration evaluation: ' + str(err))
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }
    print(f'VAL f1\t{trainer.get_best_f1()} - ({result[1][1]})')
    print(f'VAL loss\t{trainer.get_best_loss()}')

    print(f"       .---.\n \
         /     \\\n\
          \\.@-@./\n\
          /`\\_/`\\\n\
         //  _  \\\\\tLoss: {trainer.get_best_loss()}\n\
        | \\     )|_\tf1: {trainer.get_best_f1()}\n\
       /`\\_`>  <_/ \\\n\
       \\__/'---'\\__/\n")

    return {
            'loss': result[1][0],
            'status': STATUS_OK,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1(),
            'sample_iterations': trainer.get_num_samples_seen(),
            'iterations': trainer.get_num_iterations(),
            'rc': rc,
            'results': {
                'train': {
                    'loss': result[0][0],
                    'f1': result[0][1]
                },
                'validation': {
                    'loss': result[1][0],
                    'f1': result[1][1]
                },
                'test': {
                    'loss': result[2][0],
                    'f1': result[2][1]
                }
            }
        }

In [3]:
dataset_choice = 'coNLL-2003'
runs = 150
main_experiment_name = 'CoNLL-2003HyperoptTPE'
use_cuda = True
description = 'CoNLL-2003 NER TPE Hyperopt run'

if dataset_choice not in POSSIBLE_DATASET_VALUES:
    raise Error()

In [7]:
if dataset_choice == POSSIBLE_DATASET_VALUES[0]:
    PREFERENCES.defaults(
        data_root='./data/data/germeval2017',
        data_train='train_v1.4.tsv',    
        data_validation='dev_v1.4.tsv',
        data_test='test_TIMESTAMP1.tsv',
        source_index=0,
        target_vocab_index=2,
        file_format='csv',
        language='de'
    )
    from data.germeval2017 import germeval2017_dataset as dsl

    search_space = {
        'batch_size': hp.quniform('batch_size', 10, 100, 1),
        'num_encoder_blocks': hp.quniform('num_encoder_blocks', 1, 8, 1),
        'pointwise_layer_size': hp.quniform('pointwise_layer_size', 32, 256, 1),
        'clip_comments_to': hp.quniform('clip_comments_to', 10, 250, 1),
        'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.8),
        'output_dropout_rate': hp.uniform('last_layer_dropout', 0.0, 0.8),
        'num_heads': hp.choice('num_heads', [1, 2, 3, 4, 5]),
        'transformer_use_bias': hp_bool('transformer_use_bias'),
        'output_layer': hp.choice('output_layer', [
            {
                'type': OutputLayerType.Convolutions,
                'output_conv_num_filters': hp.quniform('output_conv_num_filters', 1, 400, 1),
                'output_conv_kernel_size': hp.quniform('output_conv_kernel_size', 1, 10, 1),
                'output_conv_stride': hp.quniform('output_conv_stride', 1, 10, 1),
                'output_conv_padding': hp.quniform('output_conv_padding', 0, 5, 1),
            },
            {
                'type': OutputLayerType.LinearSum
            }
        ]),
        'learning_rate_scheduler': hp.choice('learning_rate_scheduler', [
            {
                'type': LearningSchedulerType.Noam,
                'noam_learning_rate_warmup': hp.quniform('noam_learning_rate_warmup', 1000, 9000, 1),
                'noam_learning_rate_factor': hp.uniform('noam_learning_rate_factor', 0.01, 4)
            }
        ]),
        'optimizer': hp.choice('optimizer', [
            {
                'type': OptimizerType.Adam,
                'adam_beta1': hp.uniform('adam_beta1', 0.7, 0.999),
                'adam_beta2': hp.uniform('adam_beta2', 0.7, 0.999),
                'adam_eps': hp.loguniform('adam_eps', np.log(1e-10), np.log(1)),
                'learning_rate': hp.lognormal('adam_learning_rate', np.log(0.01), np.log(10)),
                'adam_weight_decay': 1*10**hp.quniform('adam_weight_decay', -8, -3, 1)
            },
            #{
            #    'type': OptimizerType.SGD,
            #    'sgd_momentum': hp.uniform('sgd_momentum', 0.4, 1),
            #    'sgd_weight_decay': hp.loguniform('sgd_weight_decay', np.log(1e-4), np.log(1)),
            #    'sgd_nesterov': hp_bool('sgd_nesterov'),
            #    'learning_rate': hp.lognormal('sgd_learning_rate', np.log(0.01), np.log(10))
        ]),
        'replace_url_tokens': hp_bool('replace_url_tokens'),
        'harmonize_bahn': hp_bool('harmonize_bahn'),
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove']),
        'embedding_name': hp.choice('embedding_name', ['6B']),
        'embedding_dim': hp.choice('embedding_dim', [300]),
        'use_stop_words': hp_bool('use_stop_words'),
        'use_spell_checker': hp_bool('use_spell_checker'),
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove']),
        'task': 'germeval'
    }

elif dataset_choice == POSSIBLE_DATASET_VALUES[1]:
     from data.organic2019 import organic_dataset as dsl
     from data.organic2019 import ORGANIC_TASK_ALL, ORGANIC_TASK_ENTITIES, ORGANIC_TASK_ATTRIBUTES, ORGANIC_TASK_ENTITIES_COMBINE, ORGANIC_TASK_COARSE
     PREFERENCES.defaults(
        data_root='./data/data/organic2019',
        data_train='train.csv',    
        data_validation='validation.csv',
        data_test='test.csv',
        source_index=0,
        target_vocab_index=1,
        file_format='csv',
        language='en'
     )

     search_space = {
        'batch_size': hp.quniform('batch_size', 10, 64, 1),
        'num_encoder_blocks': hp.quniform('num_encoder_blocks', 1, 4, 1),
        'pointwise_layer_size': hp.quniform('pointwise_layer_size', 32, 350, 1),
        'clip_comments_to': hp.quniform('clip_comments_to', 45, 180, 1),
        'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.8),
        'output_dropout_rate': hp.uniform('last_layer_dropout', 0.0, 0.8),
        'num_heads': hp.choice('num_heads', [1, 2, 3, 4, 5]),
        'transformer_use_bias': hp_bool('transformer_use_bias'),
        'output_layer': hp.choice('output_layer', [
            {
                'type': OutputLayerType.Convolutions,
                'output_conv_num_filters': hp.quniform('output_conv_num_filters', 10, 400, 1),
                'output_conv_kernel_size': hp.quniform('output_conv_kernel_size', 1, 10, 1),
                'output_conv_stride': hp.quniform('output_conv_stride', 1, 10, 1),
                'output_conv_padding': hp.quniform('output_conv_padding', 0, 5, 1),
            },
            {
                'type': OutputLayerType.LinearSum
            }
        ]),
        'learning_rate_scheduler': hp.choice('learning_rate_scheduler', [
            {
                'type': LearningSchedulerType.Noam,
                'noam_learning_rate_warmup': hp.quniform('noam_learning_rate_warmup', 1000, 9000, 1),
                'noam_learning_rate_factor': hp.uniform('noam_learning_rate_factor', 0.01, 4)
            }
        ]),
        'optimizer': hp.choice('optimizer', [
            {
                'type': OptimizerType.Adam,
                'adam_beta1': hp.uniform('adam_beta1', 0.7, 0.999),
                'adam_beta2': hp.uniform('adam_beta2', 0.7, 0.999),
                'adam_eps': hp.loguniform('adam_eps', np.log(1e-10), np.log(1)),
                'learning_rate': hp.lognormal('adam_learning_rate', np.log(0.01), np.log(10)),
                'adam_weight_decay': 1*10**hp.quniform('adam_weight_decay', -8, -3, 1)
            },
            #{
            #    'type': OptimizerType.SGD,
            #    'sgd_momentum': hp.uniform('sgd_momentum', 0.4, 1),
            #    'sgd_weight_decay': hp.loguniform('sgd_weight_decay', np.log(1e-4), np.log(1)),
            #    'sgd_nesterov': hp_bool('sgd_nesterov'),
            #    'learning_rate': hp.lognormal('sgd_learning_rate', np.log(0.01), np.log(10))
        ]),
        'task': hp.choice('task', [
            ORGANIC_TASK_ENTITIES,
            ORGANIC_TASK_ENTITIES_COMBINE
        ]),
        'use_stop_words': hp_bool('use_stop_words'),
        'use_spell_checker': hp_bool('use_spell_checker'),
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove'])
    }
elif dataset_choice == POSSIBLE_DATASET_VALUES[2]:
    PREFERENCES.defaults(
        data_root='./data/data/conll2003',
        data_train='eng.train.txt',
        data_validation='eng.testa.txt',
        data_test='eng.testb.txt',
        source_index=0,
        target_vocab_index=1,
        file_format='txt',
        language='en'
    )
    from data.conll import conll2003_dataset as dsl

    search_space = {
        'batch_size': hp.quniform('batch_size', 10, 64, 1),
        'num_encoder_blocks': hp.quniform('num_encoder_blocks', 1, 4, 1),
        'pointwise_layer_size': hp.quniform('pointwise_layer_size', 32, 350, 1),
        'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.8),
        'output_dropout_rate': hp.uniform('last_layer_dropout', 0.0, 0.8),
        'num_heads': hp.choice('num_heads', [1, 2, 3]),
        'transformer_use_bias': True,
        'learning_rate_scheduler': {
                'type': LearningSchedulerType.Noam,
                'noam_learning_rate_warmup': hp.quniform('noam_learning_rate_warmup', 1000, 9000, 1),
                'noam_learning_rate_factor': hp.uniform('noam_learning_rate_factor', 0.01, 4)
            },
        'optimizer':
            {
                'type': OptimizerType.Adam,
                'adam_beta1': 0.9,
                'adam_beta2': 0.98,
                'adam_eps': 10e-9,
                'learning_rate': hp.lognormal('adam_learning_rate', np.log(0.01), np.log(10)),
                'adam_weight_decay': 1*10**hp.quniform('adam_weight_decay', -8, -3, 1)
            },
        'output_layer': {'type': None},
        'task': 'ner',
        'clip_comments_to': 200,
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove'])
    }
    
else:
    PREFERENCES.defaults(
        data_root='./data/data/amazon/splits',
        data_train='train.pkl',    
        data_validation='val.pkl',
        data_test='test.pkl',
        source_index=0,
        target_vocab_index=1,
        file_format='pkl',
        language='en'
    )
    from data.amazon import amazon_dataset as dsl

In [8]:
experiment_name = utils.create_loggers(experiment_name=main_experiment_name)
logger = logging.getLogger(__name__)
dataset_logger = logging.getLogger('data_loader')
logger.info('Run hyper parameter random grid search for experiment with name ' + main_experiment_name)
logger.info('num_optim_iterations: ' + str(runs))

Log path is  C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\logs\CoNLL-2003HyperoptTPE\20190428\3


In [9]:
try:
    logger.info('Current commit: ' + utils.get_current_git_commit())
    print('Current commit: ' + utils.get_current_git_commit())
except Exception as err:
    logger.exception('Could not print current commit')

trials = Trials()
try:

    best = fmin(objective,
        space=search_space,
        algo=rand.suggest,
        max_evals=runs,
        trials=trials)

    print(best)
except Exception as err:
    logger.exception('Could not complete optimization')
    print('Could not complete optimization. The log file provides more details.')
    traceback.print_tb(err.__traceback__)



path = os.path.join(os.getcwd(), 'logs', f'hp_run_{main_experiment_name}.pkl')
with open(path, 'wb') as f:
    pickle.dump(trials, f)

Current commit: b'8b18e1e'
Log path is                                                                                                            
C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\logs\CoNLL-2003HyperoptTPE\20190428\4  
                                                                                                                       

#########################################################################
+----------------------------------------------------------------------------------+                                   
|                                 Hyperparameters                                  |
+------------------------------+---------------------------------------------------+
|          Parameter           |                       Value                       |
+------------------------------+---------------------------------------------------+
|            kwargs            | {'batch_size': 49.0, 'learning_rate_sche[...]

A Jupyter Widget

# EP	# IT	tr loss		val loss	f1		acc		duration / total time                                                             
1	14k	0.04		0.04		0.122		0.800		0.60m - 0.6m / 0.0m                                                                    
2	28k	0.03		0.02		0.146		0.831		0.61m - 1.3m / 21.1m                                                                   
3	42k	0.02		0.02		0.126		0.803		0.60m - 1.9m / 21.5m                                                                   
4	56k	0.01		0.02		0.118		0.797		0.61m - 2.5m / 21.1m                                                                   
5	70k	0.01		0.02		0.136		0.824		0.60m - 3.1m / 21.3m                                                                   
6	84k	0.01		0.02		0.179		0.864		0.59m - 3.7m / 21.1m                                                                   
7	98k	0.01		0.02		0.180		0.864		0.61m - 4.3m / 20.9m                                                                   
8	113k	0.01		0.02		0.176		0.858		0.59m -

A Jupyter Widget

# EP	# IT	tr loss		val loss	f1		acc		duration / total time                                                             
1	14k	0.10		0.10		0.263		0.739		0.73m - 0.7m / 0.0m                                                                    
2	28k	0.06		0.13		0.193		0.741		0.73m - 1.5m / 25.5m                                                                   
3	42k	0.05		0.11		0.252		0.770		0.73m - 2.2m / 25.6m                                                                   
4	56k	0.05		0.10		0.341		0.810		0.73m - 3.0m / 25.5m                                                                   
5	70k	0.05		0.11		0.274		0.761		0.70m - 3.7m / 25.8m                                                                   
6	84k	0.05		0.11		0.344		0.818		0.70m - 4.4m / 24.7m                                                                   
7	98k	0.05		0.11		0.319		0.802		0.74m - 5.2m / 24.9m                                                                   
8	112k	0.05		0.13		0.333		0.807		0.73m -

...

# EP	# IT	tr loss		val loss	f1		acc		duration / total time                                                             
1	14k	0.13		0.09		0.543		0.879		1.49m - 1.5m / 0.0m                                                                    
2	28k	0.06		0.09		0.465		0.851		1.43m - 2.9m / 52.0m                                                                   
3	42k	0.05		0.07		0.538		0.880		1.41m - 4.4m / 50.2m                                                                   
4	56k	0.06		0.06		0.522		0.875		1.42m - 5.8m / 49.7m                                                                   
5	70k	0.07		0.07		0.515		0.872		1.42m - 7.2m / 49.8m                                                                   
6	84k	0.08		0.07		0.486		0.858		1.44m - 8.7m / 49.8m                                                                   
VAL f1	0.5429158089937979 - (0.5429158089937979)                                                                       
VAL loss	0.06318303090792436            

A Jupyter Widget

# EP	# IT	tr loss		val loss	f1		acc		duration / total time                                                             
1	14k	0.04		0.03		0.141		0.811		0.47m - 0.5m / 0.0m                                                                    
2	28k	0.02		0.02		0.221		0.875		0.47m - 1.0m / 16.5m                                                                   
3	42k	0.01		0.01		0.237		0.885		0.47m - 1.5m / 16.6m                                                                   
4	56k	0.01		0.01		0.263		0.900		0.48m - 1.9m / 16.5m                                                                   
5	70k	0.00		0.01		0.229		0.881		0.47m - 2.4m / 16.8m                                                                   
6	84k	0.00		0.01		0.256		0.895		0.50m - 2.9m / 16.4m                                                                   
7	98k	0.00		0.01		0.268		0.900		0.48m - 3.4m / 17.4m                                                                   
8	112k	0.00		0.01		0.264		0.899		0.49m -

A Jupyter Widget

# EP	# IT	tr loss		val loss	f1		acc		duration / total time                                                             
1	14k	0.03		0.01		0.166		0.842		0.76m - 0.8m / 0.0m                                                                    
2	28k	0.01		0.01		0.201		0.869		0.76m - 1.5m / 26.6m                                                                   
3	42k	0.01		0.01		0.227		0.888		0.75m - 2.3m / 26.5m                                                                   
4	56k	0.00		0.01		0.243		0.895		0.76m - 3.1m / 26.5m                                                                   
5	70k	0.01		0.01		0.202		0.874		0.75m - 3.9m / 26.7m                                                                   
6	84k	0.01		0.01		0.196		0.873		0.76m - 4.6m / 26.3m                                                                   
7	99k	0.01		0.02		0.063		0.734		0.74m - 5.4m / 26.6m                                                                   
8	113k	0.02		0.02		0.104		0.762		0.75m -